In [1]:
import matplotlib.pyplot as plt
from PIL import Image


def latex_to_image(latex_str, filename="output.png"):
    """
    Renders a LaTeX expression to an image with a white background.

    Args:
    latex_str (str): The LaTeX string to render.
    filename (str): The filename where the image will be saved.
    """
    # Configure Matplotlib: use white background and remove axes
    plt.rcParams.update(
        {
            "text.usetex": True,
            "text.latex.preamble": r"\usepackage{amsmath}",
            "axes.facecolor": "white",
            "axes.edgecolor": "white",
            "axes.linewidth": 0,
            "xtick.bottom": False,
            "xtick.top": False,
            "ytick.left": False,
            "ytick.right": False,
            "xtick.labelbottom": False,
            "xtick.labeltop": False,
            "ytick.labelleft": False,
            "ytick.labelright": False,
        }
    )
    plt.figure(figsize=(2, 1))
    plt.text(
        0, 0, f"${latex_str}$", fontsize=40, ha="center", va="center", color="white"
    )
    plt.axis("off")
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1.5, bottom=0, right=0.5, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    # Plot color is white
    plt.gca().set_facecolor("white")

    # Save the image
    plt.savefig(filename, bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

    # Open the image and invert the colors, then save it

    img = Image.open(filename)
    img = img.convert("L")
    img = img.point(lambda p: 255 - p)
    img.save(filename)


# Test
latex_to_image(r"\frac{1}{2} \cdot \frac{3}{4} = \frac{3}{8}", "output.png")

In [2]:
# Generate arbitrary LaTeX strings

# Singletons
numbers = [str(i) for i in range(100)]
# Alphabet
alphabet = [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)]
# Greek alphabet LaTeX symbols (with \)
greek_lower = [
    "\\alpha",
    "\\beta",
    "\\gamma",
    "\\delta",
    "\\epsilon",
    "\\zeta",
    "\\eta",
    "\\theta",
    "\\iota",
    "\\kappa",
    "\\lambda",
    "\\mu",
    "\\nu",
    "\\xi",
    "\\omicron",
    "\\pi",
    "\\rho",
    "\\sigma",
    "\\tau",
    "\\upsilon",
    "\\phi",
    "\\chi",
    "\\psi",
    "\\omega",
]

greek_lower = ["{" + ch + "}" for ch in greek_lower]

special_singleton = [
    "{\\infty}",
    "{\\emptyset}",
]

# Two element operators
two_op = {
    "#+#": 1,
    "#-#": 1,
    "#\\cdot #": 1,
    "#\\times #": 1,
    "##": 4,
    "#/#": 2,
    "\\frac{#}{#}": 3,
    "{#}^{#}": 3,
    "{#}_{#}": 3,
}

# One element operators
one_op = {
    "-#": 2,
    "\\sum_{i=0}^{n}#": 0.2,
    "\\sum_{n=1}^{\\infty}#": 0.2,
    "\\int_{-\\infty}^{\\infty}#": 0.2,
    "\\int_{0}^{1}#": 0.2,
    "\\sqrt{#}": 1,
    "\\sin{#}": 1,
    "\\cos{#}": 1,
    "\\tan{#}": 1,
    "\\sec{#}": 1,
    "\\csc{#}": 1,
    "\\cot{#}": 1,
}

# Three element operators
three_op = {"\\int_{#}^{#}#": 3, "\\sum_{#}^{#}#": 2, "###": 4}

singletons = numbers + alphabet + greek_lower + special_singleton
one_ops = one_op
two_ops = two_op
three_ops = three_op

In [3]:
# Construct a random LaTeX expression
import random


def sample_from_dict(d):
    # Weights are not normalized
    total = sum(d.values())
    rand = random.random() * total
    for k, v in d.items():
        rand -= v
        if rand <= 0:
            return k
    return k


# Methodology: Start with a #. Now until it has no more #, replace the first # with an expresson. Expression is singleton with probablity 0.5, one_op with probability 0.3, two_op with probability 0.1, three_op with probability 0.1
def generate_expression():
    expression = "#"
    while "#" in expression:
        rand = random.random()
        expressions = [
            random.choice(singletons),
            sample_from_dict(one_ops),
            sample_from_dict(two_ops),
            sample_from_dict(three_ops),
        ]

        if rand < 0.6:
            expression = expression.replace("#", expressions[0], 1)
        elif rand < 0.7:
            expression = expression.replace("#", expressions[1], 1)
        elif rand < 0.95:
            expression = expression.replace("#", expressions[2], 1)
        else:
            expression = expression.replace("#", expressions[3], 1)
    return expression


# expression = ""
# for i in range(10):
#     while len(expression) < 400:
#         expression = generate_expression()
#     print("$" + expression + "$")
#     expression = ""

In [4]:
f = open("dataset/latex.txt", "w")


def gen_exp_and_save(min_len, max_len):
    expression = ""
    while len(expression) <= min_len or len(expression) > max_len:
        expression = generate_expression()
    print(expression)
    latex_to_image(expression, filename=f"dataset/{i}.png")
    f.write(expression + "\n")


for i in range(1000):
    desired_min_length = {
        0: 0,
        200: 5,
        400: 10,
        600: 15,
        800: 20,
        1000: 25,
        1200: 30,
    }
    i_round = (i // 200) * 200
    success = False
    while not success:
        try:
            gen_exp_and_save(
                desired_min_length[i_round], desired_min_length[i_round + 200]
            )
            success = True
        except:
            pass

    # If i is multiple of 200, save f
    if i % 200 == 0:
        f.close()
        f = open("dataset/latex.txt", "a")

f.close()

85
Z
-38
50
f
17
U
9
65
46
70
8
64
21
40
e
77
72
b
A
W
72
S
v
c
F
75
O
28
I
31
65
95
17
{\xi}
l
34
53
94
92
56
e
54
0
W
C
E
P
10
49
w
94
Z
h
49
c
I/S
82
D
54
D
O
14
W
r
6
M
l/3
60
I
50
72
70
81
R
78
p
34
G
C
--17
97+91
60
87
80
60
w
a
13
41
81
61
i4630
87
11
32
q+59
44
-91
75
D
91
74
W
o
L
16
39
I
B
24
O
27
58
R-S
42
97
d
5823
e
63
10
t
39
87
3
7
19
75
F
y
16
77
31
25/73
21
98
90
N
b
g
V
56
98
-1
T
d
12
97
30
38
90
90
QR
a
1880
-11
84
83
17
7
r
79
{\xi}
H
a
46
g
66
35
23
78
66
81
61
39
U
79
V/37
w
65
K
96
1
1
76
X+91
18
53
V
36
41
23
{\nu}
29
57
F
p
61
5
{\lambda}
{\upsilon}
{\phi}
{\psi}
{96}^{88}
{\delta}
{94}^{91}
o/w\tan{X}
\tan{42}o
-{\omega}
{72}_{72}
{17}_{66}
{\sigma}
{1}^{31}
{\omega}
R\cdot 95
{\delta}
{\omega}
{\beta}
{\kappa}
\cos{R}
-\cos{61}
Z\times G
-\sin{98}
29\times q
{\omicron}
{80}^{52}
{\beta}/7
{\upsilon}
{\chi}
{\zeta}/98
\csc{E}
{\upsilon}
{\upsilon}
-\sin{x}
{70}_{94}
{\zeta}
{X}^{8}
\cot{8}
-{\xi}
\tan{46}
{\iota}
{\zeta}
{\iota}
{63}^{91}
\csc{18}
-{\psi}
{\e

RuntimeError: latex was not able to process the following string:
b'${\\\\omicron}$'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp87gqyonv cfa1d053ece6b1dae2043724ccd31142.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./cfa1d053ece6b1dae2043724ccd31142.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/2023/texmf-dist

RuntimeError: latex was not able to process the following string:
b'${\\\\omicron}$'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpsmd93mg2 cfa1d053ece6b1dae2043724ccd31142.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./cfa1d053ece6b1dae2043724ccd31142.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/2023/texmf-dist

<Figure size 200x100 with 1 Axes>

RuntimeError: latex was not able to process the following string:
b'${\\\\omicron}$'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpe9qu19bs cfa1d053ece6b1dae2043724ccd31142.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./cfa1d053ece6b1dae2043724ccd31142.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/2023/texmf-dist

...

<Figure size 200x100 with 1 Axes>

RuntimeError: latex was not able to process the following string:
b'${\\\\sin{{\\\\omicron}}}_{C/97}$'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpe92j3fx7 757280ec2c8a4c84288d9e4db5544a45.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./757280ec2c8a4c84288d9e4db5544a45.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texli

<Figure size 200x100 with 1 Axes>

RuntimeError: latex was not able to process the following string:
b'$\\\\cos{{\\\\omicron}}\\\\cdot 79$'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp5ajheds4 2e69dd547fcf50ef402362faf8d64f5d.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./2e69dd547fcf50ef402362faf8d64f5d.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/tex

<Figure size 200x100 with 1 Axes>